In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim import utils
import gensim.downloader as api
import gensim.models
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import classification_report

In [3]:
data = pd.read_csv(r"amazon_reviews_us_Beauty_v1_00.tsv", sep = '\t', error_bad_lines=False,usecols = ['star_rating', 'review_body'] )

C:\Users\HP\anaconda3\envs\tensorflow_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
class_1 = data.loc[data['star_rating'].isin(["1", "2"])]
class_2 = data.loc[data['star_rating'].isin(["3"])]
class_3 = data.loc[data['star_rating'].isin(["4", "5"])]

In [5]:
class_1 = class_1.sample(n = 20000, random_state = 2)
class_2 = class_2.sample(n = 20000, random_state = 2)
class_3 = class_3.sample(n = 20000, random_state = 2)

In [6]:
class_1['output_class'] = '1'
class_2['output_class'] = '2'
class_3['output_class'] = '3'

In [7]:
complete_data = class_1.append(class_2)
complete_data = complete_data.append(class_3)

In [8]:
complete_data = complete_data.reset_index(drop = True)

In [9]:
complete_data

,star_rating,review_body,output_class
0,1,"Hmmm, where do I start? First of all I have be...",1
1,1,Don't buy these dull pathetic shears. Tried t...,1
2,1,Did the company change the formula? This is my...,1
3,1,leaky bottle in shipment. so many people sell ...,1
4,2,I saw this and thought it'd be worth a try; ho...,1
...,...,...,...
59995,5,Excellent product. Works great on my skin.,3
59996,5,Great price - great blades.,3
59997,5,This hair was AWESOME! It wasn't too thick/ po...,3
59998,5,My wife loves the product and actually put som...,3


### 2. Word Embedding (25 points)

### (a) Load the pretrained Google News word2vec model and learn how to extract word embeddings for your dataset.  

In [10]:
wv = api.load('word2vec-google-news-300')

In [62]:
vector1 = wv['king'] - wv['man'] + wv['woman']
vector2 = wv['queen']

cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

print(cosine_similarity)

0.73005176


In [63]:
vector1 = wv['excellent']
vector2 = wv['outstanding']

cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

print(cosine_similarity)

0.5567486


In [12]:
pairs = [
    ('excellent', 'outstanding'),  
    ('king', 'queen'),
    ('woman', 'queen'),
   
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'excellent'	'outstanding'	0.56
'king'	'queen'	0.65
'woman'	'queen'	0.32


### (b) Train a Word2Vec model using your own dataset. You will use these extracted features in the subsequent questions of this assignment. Set the embedding size to be 300 and the window size to be 13. You can also consider a minimum word count of 9. Check the semantic similarities for the same two examples in part (a). What do you conclude from comparing vectors generated by yourself and the pretrained model? Which of the Word2Vec models seems to encode semantic similarities between words better? For the rest of this assignment, use the pretrained “word2vec-google news-300” Word2Vec features.


In [13]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = data['review_body'].dropna()
        for line in corpus_path:
#             print(line)
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [54]:
sentences = MyCorpus()
model_w2v = gensim.models.Word2Vec(
    sentences=sentences,
    vector_size=300,
    min_count=9,
    window = 13
)

In [60]:
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(model_w2v.wv["king"].reshape(-1, 1), model_w2v.wv["queen"].reshape(-1, 1))
vector1 = model_w2v.wv["king"] - model_w2v.wv["man"] + model_w2v.wv["woman"]
vector2 =  model_w2v.wv["queen"]
cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

print(cosine_similarity)

0.3874093


In [66]:
vector1 = model_w2v.wv["excellent"]
vector2 =  model_w2v.wv["outstanding"]
cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

print(cosine_similarity)

0.83489233


### Q) What do you conclude from comparing vectors generated by yourself and the pretrained model? Which of the Word2Vec models seems to encode semantic similarities between words better?

### A) Comparing the vectors generated from the pretrained model and my own model, I observed that the pretrained model encodes the semantic similarities between the words better than my model in general. The reason can the be the dataset used for the pretrained model is huge.

#### King − Man + Woman = Queen
Similarity for Google Word2Vec - 0.73005176

Similarity for My Model - 0.3874093

#### Outstanding ~ Excellent

Similarity for Google Word2Vec - 0.5567486

Similarity for My Model - 0.83489233

#### For the example Outstanding ~ Excellent our model performs better. The reason might be that our dataset consists of reviews, in which these words are very common and indicate the same sentiment. This semantic similarity was captured well by our model.

### 3. Simple models (20 points)


### Using the Google pre-trained Word2Vec features, train a single perceptron and an SVM model for the classification problem. For this purpose, use the average Word2Vec vectors for each review as the input feature.

In [17]:
def extract_features(data):

    data['review_body'] = data['review_body'].dropna().apply(lambda x: x.lower())
    data['review_body'] = data['review_body'].dropna().apply(lambda x: x.split())
    data['review_body'] = data['review_body'].dropna().apply(lambda x: [word for word in x if word in wv])
    data['review_body'] = data['review_body'].dropna().apply(lambda x: np.mean([wv[word] for word in x], axis = 0))
    data.dropna(subset=['review_body'], inplace=True)

    X = np.vstack(data['review_body'].values)
    y = data['output_class'].values
    
    
    return X, y

In [18]:
single_perceptron_data = complete_data.copy()

In [19]:
X, y = extract_features(single_perceptron_data)

C:\Users\HP\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Define the perceptron model

perceptron = Perceptron()

perceptron.fit(X_train, y_train)

# Predict labels for the test data
y_pred = perceptron.predict(X_test)

# Evaluate the model's performance

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [22]:
print("Perceptron results from word2vec embeddings")
print("Accuracy:", accuracy)
print("Precision", precision)
print("Recall", recall)
print("F1", f1)

Perceptron results from word2vec embeddings
Accuracy: 0.5959180245254494
Precision 0.6364002047528308
Recall 0.5959180245254494
F1 0.5890829957518432


In [28]:
svm_model = svm.SVC(kernel='linear')

# Train the model on the training data
svm_model.fit(X_train, y_train)

# Predict labels for the test data
y_pred = svm_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [29]:
print("SVM results from word2vec embeddings")
print("Accuracy:", accuracy)
print("Precision", precision)
print("Recall", recall)
print("F1", f1)

SVM results from word2vec embeddings
Accuracy: 0.6500923903913993
Precision 0.6574941067400007
Recall 0.6500923903913993
F1 0.6526496371540883


### Q) What do you conclude from comparing performances for the models trained using the two different feature types 

### In HW1,  the TF-IDF embedding used generated the following results on test data:
1.  Accuracy for Perceptron - 0.6070
2. Accuracy for SVM - 0.65

### On using Word2Vec embeddings in this assignment the results generated are as follows:
1. Accuracy for Perceptron - 0.5959
2. Accuracy for SVM - 0.65

### On comparing both these results, we can say that the results are pretty similar for both the embeddings. Both thes embeddings give very similar results.

### 4. Feedforward Neural Networks (25 points)


In [30]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### (a) (10 points)
To generate the input features, use the average Word2Vec vectors similar to
the “Simple models” section and train the neural network. Report accuracy
values on the testing split for your MLP.


In [31]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100,  input_shape=(300,)),
    tf.keras.layers.Dense(10 ),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model with cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the model with a batch size of 32 and for 10 epochs
model.fit(X_train, y_train, batch_size=32, epochs=20)


# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Accuracy and Loss for test data for FNN using word2vec embeddings.")
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


Epoch 1/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8669 - accuracy: 0.6102
Epoch 2/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8330 - accuracy: 0.6376
Epoch 3/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8267 - accuracy: 0.6411
Epoch 4/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8223 - accuracy: 0.6474
Epoch 5/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8216 - accuracy: 0.6462
Epoch 6/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8198 - accuracy: 0.6467
Epoch 7/20
1489/1489 [==============================] - 2s 1ms/step - loss: 0.8196 - accuracy: 0.6469
Epoch 8/20
1489/1489 [==============================] - 1s 977us/step - loss: 0.8188 - accuracy: 0.6488
Epoch 9/20
1489/1489 [==============================] - 1s 986us/step - loss: 0.8184 - accuracy: 0.6488
Epoch 10/20
1489/1489 [==============================] - 1s 994us/step - loss:

### (b) (15 points)
To generate the input features, concatenate the first 10 Word2Vec vectors
for each review as the input feature (x = [WT
1
, ..., WT
10]) and train the neural
network. Report the accuracy value on the testing split for your MLP model.
What do you conclude by comparing accuracy values you obtain with
those obtained in the “’Simple Models” section.

In [35]:
def get_sentence_embeddings(df, model, num_words=10):

    embeddings = []
    for sentence in df['review_body']:
        
        if type(sentence) != float:
            words = sentence.split()[:num_words]  # Take the first num_words words in the sentence
            sentence_embeddings = []
            for word in words:
                if word in model:
                    sentence_embeddings.append(model[word])
                else:
                    sentence_embeddings.append(np.zeros(300))
            # Pad the sentence_embeddings list with zero vectors if necessary
            while len(sentence_embeddings) < num_words:
                sentence_embeddings.append(np.zeros(300))
            
            embeddings.append(np.concatenate(sentence_embeddings))
        else:
            sentence_embeddings = []
            while len(sentence_embeddings) < num_words:
                sentence_embeddings.append(np.zeros(300))
            
            embeddings.append(np.concatenate(sentence_embeddings))
            
    return np.array(embeddings)

In [36]:
muti_percp_data = complete_data.copy()

In [38]:
data_array = get_sentence_embeddings(muti_percp_data, wv, 10)

In [40]:
X_10 = data_array

In [41]:
y_10 = np.array(complete_data['output_class'])

In [42]:
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=42)

In [43]:
# Take 10 words.
y_train_10 = to_categorical(y_train_10)
y_test_10 = to_categorical(y_test_10)

In [44]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100,  input_shape=(3000,)),
    tf.keras.layers.Dense(10 ),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model with cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the model with a batch size of 32 and for 10 epochs
model.fit(X_train_10, y_train_10, batch_size=32, epochs=20)


# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_10, y_test_10)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


Epoch 1/20
1500/1500 [==============================] - 6s 3ms/step - loss: 0.9123 - accuracy: 0.5654
Epoch 2/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.8565 - accuracy: 0.6011
Epoch 3/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.8410 - accuracy: 0.6132
Epoch 4/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.8318 - accuracy: 0.6196
Epoch 5/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.8255 - accuracy: 0.6228
Epoch 6/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.8208 - accuracy: 0.6253
Epoch 7/20
1500/1500 [==============================] - 5s 4ms/step - loss: 0.8159 - accuracy: 0.6285
Epoch 8/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.8125 - accuracy: 0.6298
Epoch 9/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.8090 - accuracy: 0.6326
Epoch 10/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.807

### Q) What do you conclude by comparing accuracy values you obtain with those obtained in the “’Simple Models” section.

### A) On comparing the performance of Simple Models to FNN models, we can observe that the on the entire dataset, Feedforward Neural Network outperforms the Perceptron Model and performs similar to SVM. With more tuning and epochs, the FNN might have an edge over both the Simple Models. FNN capture the complex relations better as they have non-linear decision  boundaries. As the features are well crafted, they perform almost similarly. 

### The accuracies are as follows:
1. Perceptron - 0.5959180245254494
2. SVM - 0.6500923903913993
3. FNN(All the words) - 0.6479926109313965
4. FNN(For 10 words) - 0.5947499871253967

### We can also see that the FNN with average of vectors of all the words in the review performs better than FNN for concatenated 10 words.

### 5. Recurrent Neural Networks (30 points)

### (a) (10 points)
Train a simple RNN for sentiment analysis. You can consider an RNN cell
with the hidden state size of 20. To feed your data into our RNN, limit
the maximum review length to 20 by truncating longer reviews and padding
shorter reviews with a null value (0). Report accuracy values on the testing
split for your RNN model.
What do you conclude by comparing accuracy values you obtain with
those obtained with feedforward neural network models.

In [45]:
def get_sentence_embeddings_RNN(df, model, num_words=10):
    # Get the embeddings for each sentence in the dataframe
    # df.dropna(subset=['review_body'], inplace=True)

    embeddings = []
    for sentence in df:
        if type(sentence) != float:
            words = sentence.split()[:num_words]  # Take the first num_words words in the sentence
            sentence_embeddings = []
            for word in words:
                if word in model:
                    sentence_embeddings.append(model[word])
                else:
                    sentence_embeddings.append(np.zeros(300))
          # Pad the sentence_embeddings list with zero vectors if necessary
            while len(sentence_embeddings) < num_words:
                sentence_embeddings.append(np.zeros(300))
            embeddings.append(sentence_embeddings)
        else:
            sentence_embeddings = []
            while len(sentence_embeddings) < num_words:
                sentence_embeddings.append(np.zeros(300))
            embeddings.append(sentence_embeddings)
    return np.array(embeddings)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(complete_data['review_body'], complete_data['output_class'], test_size=0.2, random_state=42)

In [49]:
X_train = get_sentence_embeddings_RNN(X_train, wv, 20)
X_test = get_sentence_embeddings_RNN(X_test, wv, 20)

In [50]:
y_train= to_categorical(y_train)
y_test = to_categorical(y_test)

In [51]:
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(20, input_shape=(20, 300)),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=32, epochs=20)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/20
1500/1500 [==============================] - 10s 5ms/step - loss: 0.9394 - accuracy: 0.5364
Epoch 2/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.8652 - accuracy: 0.5840
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8497 - accuracy: 0.5919
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8380 - accuracy: 0.6030
Epoch 5/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8260 - accuracy: 0.6164
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8196 - accuracy: 0.6207
Epoch 7/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.8117 - accuracy: 0.6261
Epoch 8/20
1500/1500 [==============================] - 7s 4ms/step - loss: 0.8062 - accuracy: 0.6301
Epoch 9/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8013 - accuracy: 0.6345
Epoch 10/20
1500/1500 [==============================] - 7s 4ms/step - loss: 0.79

### Q) What do you conclude by comparing accuracy values you obtain with those obtained with feedforward neural network models.

### A) The test accuracies are as follows:
1. RNN(For 20 words of the reviews) - 0.6162499785423279
2. FNN(For entire corpus) - 0.6479926109313965
3. FNN(For 10 words) - 0.5947499871253967

### From these results we can see that the FNN trained on entire dataset outperforms the FNN(For 10 words) and RNN(For 20 words). As the RNN is not trained on the entire corpus, but still manages to get an accuracy close to FNN, with more data it might perform better. 

### FNN can handle large data better. Meanwhile, RNN can perform better on complex data as they have a memory component.

### (b) (10 points)
Repeat part (a) by considering a gated recurrent unit cell.

In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20, 300)),
    tf.keras.layers.GRU(20),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=32, epochs=20)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/20
1500/1500 [==============================] - 11s 6ms/step - loss: 0.9136 - accuracy: 0.5491
Epoch 2/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7884 - accuracy: 0.6414
Epoch 3/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7623 - accuracy: 0.6560
Epoch 4/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7440 - accuracy: 0.6670: 0s - l
Epoch 5/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7278 - accuracy: 0.6741
Epoch 6/20
1500/1500 [==============================] - 8s 5ms/step - loss: 0.7159 - accuracy: 0.6816
Epoch 7/20
1500/1500 [==============================] - 8s 6ms/step - loss: 0.7044 - accuracy: 0.6877
Epoch 8/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.6956 - accuracy: 0.6920
Epoch 9/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.6851 - accuracy: 0.6972
Epoch 10/20
1500/1500 [==============================] - 10s 7ms/step - l

### (c) (10 points)
Repeat part (a) by considering an LSTM unit cell.


In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20, 300)),
    tf.keras.layers.LSTM(20),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=32, epochs=20)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/20
1500/1500 [==============================] - 11s 6ms/step - loss: 0.9126 - accuracy: 0.5502
Epoch 2/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.7988 - accuracy: 0.6355
Epoch 3/20
1500/1500 [==============================] - 10s 7ms/step - loss: 0.7701 - accuracy: 0.6516
Epoch 4/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.7529 - accuracy: 0.6617
Epoch 5/20
1500/1500 [==============================] - 13s 9ms/step - loss: 0.7379 - accuracy: 0.6697
Epoch 6/20
1500/1500 [==============================] - 11s 8ms/step - loss: 0.7251 - accuracy: 0.6758
Epoch 7/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.7127 - accuracy: 0.6825
Epoch 8/20
1500/1500 [==============================] - 10s 6ms/step - loss: 0.7006 - accuracy: 0.6902
Epoch 9/20
1500/1500 [==============================] - 9s 6ms/step - loss: 0.6884 - accuracy: 0.6961
Epoch 10/20
1500/1500 [==============================] - 9s 6ms/step - loss

### Q) What do you conclude by comparing accuracy values you obtain by GRU, LSTM, and simple RNN.

### A) On comparing the accuracies between GRU, LSTM and simple RNN, I observed that the LSTM and GRU perform better than the Simple RNN. The test accuracy between GRU and LSTM is almost the same. LSTM outperforms by 0.01. LSTM retain memory for longer.

#### The accuracies are as follows:
1. Simple RNN - 0.6162499785423279
2. GRU - 0.6643333435058594
3. LSTM - 0.6569166779518127

References: 
    https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html
    https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    https://www.tensorflow.org/tutorials/quickstart/beginner
   